In [1]:
from sklearn.datasets import make_regression
from sklearn.svm import SVR

from shapiq.explainer.product_kernel.conversion import convert_svm
from shapiq.explainer.product_kernel.game import ProductKernelGame

# Regression

In [2]:
X, y = make_regression(n_samples=100, n_features=2, noise=0.1)

In [3]:
svm_model = SVR(kernel="rbf", C=1.0, epsilon=0.1)
svm_model.fit(X, y)

SVR()

## Conversion 

In [4]:
converted_svm = convert_svm(svm_model)

## ProductKernelGame

In [5]:
kernel_game = ProductKernelGame(
    n_players=svm_model.n_features_in_, explain_point=X[0], model=converted_svm, normalize=True
)

## Exact Shapley Computation

In [6]:
from shapiq.game_theory.exact import ExactComputer

exact_computer = ExactComputer(n_players=kernel_game.n_players, game=kernel_game)

In [7]:
sv_values = exact_computer("SV").values
sv_values

array([ -5.53529552,   3.19675785, -14.51217156])

## Shapley vs Prediction

In [8]:
svm_model.predict(X[0].reshape(1, -1)), sv_values.sum()

(array([-16.85070923]), np.float64(-16.85070922902086))

# Classification

In [9]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC

In [10]:
X, y = make_classification(
    n_samples=100, n_features=2, n_informative=2, n_redundant=0, random_state=42
)
clf = SVC(kernel="rbf", gamma="scale", random_state=42)
clf.fit(X, y)

SVC(random_state=42)

## Conversion

In [11]:
converted_svc = convert_svm(clf)

## Product Kernel Game

In [12]:
point_to_explain = X[0]

kernel_game = ProductKernelGame(
    n_players=2, explain_point=point_to_explain, model=converted_svc, normalize=True
)

## Exact Shapley Computation

In [13]:
exact_computer = ExactComputer(n_players=kernel_game.n_players, game=kernel_game)
sv_values = exact_computer("SV").values

In [14]:
sv_values

array([-0.06888465,  0.52342623, -1.16634168])

## Prediction vs Shapley Calculation Comparision

In [15]:
decision_value = clf.decision_function(point_to_explain.reshape(1, -1))
decision_value, sv_values.sum()

(array([-0.7118001]), np.float64(-0.7118000987280122))